In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('stack-overflow-data.csv')


In [3]:
data.head()

,post,tags
0,what is causing this behavior in our c# datet...,c#
1,have dynamic html load as if it was in an ifra...,asp.net
2,how to convert a float value in to min:sec i ...,objective-c
3,.net framework 4 redistributable just wonderi...,.net
4,trying to calculate and print the mean and its...,python


In [4]:
print("Input data has {} rows and {} columns".format(len(data), len(data.columns)))

Input data has 40000 rows and 2 columns


In [5]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import string
from bs4 import BeautifulSoup

stopwords = nltk.corpus.stopwords.words('english')
wn = nltk.WordNetLemmatizer()
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')

In [6]:
def clean_text(text):
    text = BeautifulSoup(text, "lxml").text
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    text = REPLACE_BY_SPACE_RE.sub(' ', text) 
    text = BAD_SYMBOLS_RE.sub('', text) 
    tokens = re.split('\W+', text)
    text = [wn.lemmatize(word) for word in tokens if word not in stopwords]
    return text

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data['post'], data['tags'], test_size=0.2)

In [8]:
tfidf_vect = TfidfVectorizer(analyzer=clean_text, min_df=2, encoding='latin-1', ngram_range=(1, 2))
tfidf_vect_fit = tfidf_vect.fit(X_train)

tfidf_train = tfidf_vect_fit.transform(X_train)
tfidf_test = tfidf_vect_fit.transform(X_test)

X_train_vect = pd.DataFrame(tfidf_train.toarray())
X_test_vect = pd.DataFrame(tfidf_test.toarray())
X_train_vect.head()

,0,1,2,3,4,5,6,7,8,9,...,42211,42212,42213,42214,42215,42216,42217,42218,42219,42220
0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.046754,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import precision_recall_fscore_support as score
import time

In [ ]:
sgd = SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)

sgd_model = sgd.fit(X_train_vect, y_train)
y_pred = sgd_model.predict(X_test_vect)

precision, recall, fscore, train_support = score(y_test, y_pred)
print('Precision: {}'.format(precision))
print('Recall: {}'.format(recall))
accuracy = ((y_pred==y_test).sum()/len(y_pred))
print('Accuracy: {}'.format(accuracy))
    